In [ ]:
!pip install transformers huggingface accelerate datasets pydub tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

# The Goal of this notebook is use whisper to help my collect audio data for arabic sentence detection
1. Web scraping to get the urls for the words i need
2. Remove the duplicates from the Youtube URLs.
3. Use Whisper to collect this data and then clip it with knowing the timestamp of the word
4. clean everything and make the url check done also remove the downloaded youtube and audio add 0.5 second above and after the word to get it right
5. manually clean this data

# Web Scraping Code

In [ ]:
# from pytube import Search
# s = Search('YouTube Rewind')
# len(s.results)

In [2]:
import pandas as pd
df= pd.read_csv("youtube_urls.csv")
# df["checked"] = 0 #! To Mark if the video has been downloaded before or not 
df.head()

,id,type,title,publishedTime,duration,viewCount,url
0,qmCHCMlEvQ8,video,$kinny - Never Snitch (Official Video) سكيني -...,3 years ago,2:13,24M views,https://www.youtube.com/watch?v=qmCHCMlEvQ8
1,qmCHCMlEvQ8,video,$kinny - Never Snitch (Official Video) سكيني -...,3 years ago,2:13,"24,879,443 views",https://www.youtube.com/watch?v=qmCHCMlEvQ8
2,TLpiGpcJZ8E,video,الحمد لله ♥️🙏 | فارس قطرية - Fares katrya,4 weeks ago,3:12,55K views,https://www.youtube.com/watch?v=TLpiGpcJZ8E
3,TLpiGpcJZ8E,video,الحمد لله ♥️🙏 | فارس قطرية - Fares katrya,4 weeks ago,3:12,"55,991 views",https://www.youtube.com/watch?v=TLpiGpcJZ8E
4,P2BmrLE2QSI,video,فيلم الكوميديا والمغامرة - 1000 حمدالله ع السل...,1 month ago,1:52:54,2.1M views,https://www.youtube.com/watch?v=P2BmrLE2QSI


In [5]:
def parse_duration(duration_str):
    try:
        if isinstance(duration_str, str):
            parts = duration_str.split(':')
            if len(parts) == 2:
                hours, minutes, seconds = 0, int(parts[0]), int(parts[1])
            elif len(parts) == 3:
                hours, minutes, seconds = int(parts[0]), int(parts[1]), int(parts[2])
            else:
                raise ValueError(f"Invalid duration format: {duration_str}")

            return pd.Timedelta(hours=hours, minutes=minutes, seconds=seconds)
        else:
            return pd.NaT
    except ValueError:
        return pd.NaT

df['duration'] = df['duration'].apply(parse_duration)
filtered_df = df[df['duration'] < pd.Timedelta(minutes=15)]

filtered_df.head()

,id,type,title,publishedTime,duration,viewCount,url
0,qmCHCMlEvQ8,video,$kinny - Never Snitch (Official Video) سكيني -...,3 years ago,0 days 00:02:13,24M views,https://www.youtube.com/watch?v=qmCHCMlEvQ8
1,qmCHCMlEvQ8,video,$kinny - Never Snitch (Official Video) سكيني -...,3 years ago,0 days 00:02:13,"24,879,443 views",https://www.youtube.com/watch?v=qmCHCMlEvQ8
2,TLpiGpcJZ8E,video,الحمد لله ♥️🙏 | فارس قطرية - Fares katrya,4 weeks ago,0 days 00:03:12,55K views,https://www.youtube.com/watch?v=TLpiGpcJZ8E
3,TLpiGpcJZ8E,video,الحمد لله ♥️🙏 | فارس قطرية - Fares katrya,4 weeks ago,0 days 00:03:12,"55,991 views",https://www.youtube.com/watch?v=TLpiGpcJZ8E
6,4CSUrZFwVVI,video,Sharmoofers - Al Hamdullah | Official Lyrics V...,3 months ago,0 days 00:03:30,357K views,https://www.youtube.com/watch?v=4CSUrZFwVVI


In [6]:
filtered_df = filtered_df.drop_duplicates()

## Download the Youtube URLS and MAKE them as downloaded completed

In [ ]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pytube-15.0.0-py3-none-any.whl (57 kB)


In [3]:
from pytube import YouTube
YouTube('https://youtu.be/9bZkp7q19f0').streams.first().download()
yt = YouTube('http://youtube.com/watch?v=9bZkp7q19f0')
yt.streams


[<Stream: itag="17" mime_type="video/3gpp" res="144p" fps="12fps" vcodec="mp4v.20.3" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="18" mime_type="video/mp4" res="360p" fps="24fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="24fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="24fps" vcodec="avc1.640028" progressive="False" type="video">, <Stream: itag="216" mime_type="video/mp4" res="None" fps="1fps" vcodec="avc1.640028" progressive="False" type="video">, <Stream: itag="248" mime_type="video/webm" res="1080p" fps="24fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="136" mime_type="video/mp4" res="720p" fps="24fps" vcodec="avc1.4d401f" progressive="False" type="video">, <Stream: itag="214" mime_type="video/mp4" res="None" fps="1fps" vcodec="avc1.4d401f" progressive="False" typ

In [9]:
import pytube as pt
import os
print(pt.__version__)
yt = pt.YouTube('https://www.youtube.com/watch?v=_brWPpALvBo')
print(yt.streams)

15.0.0
[<Stream: itag="17" mime_type="video/3gpp" res="144p" fps="6fps" vcodec="mp4v.20.3" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="18" mime_type="video/mp4" res="360p" fps="25fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="25fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="136" mime_type="video/mp4" res="720p" fps="25fps" vcodec="avc1.4d401f" progressive="False" type="video">, <Stream: itag="247" mime_type="video/webm" res="720p" fps="25fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="135" mime_type="video/mp4" res="480p" fps="25fps" vcodec="avc1.4d401e" progressive="False" type="video">, <Stream: itag="244" mime_type="video/webm" res="480p" fps="25fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="134" mime_type="video/mp4" res="360p" fps="25fps" vcodec="avc1.4d401e" progressive="False" type

In [ ]:
# !pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.8 MB/s eta 0:00:00


In [8]:
import pytube
for index, row in filtered_df.iterrows():
    url = row["url"]
    filename = row["id"]
    yt = pytube.YouTube(url)
    yt.streams.filter(abr="160kbps",progressive=True).first().download(filename=filename)

AttributeError: 'NoneType' object has no attribute 'download'

### Imports and Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# USE Whisper fine-tuned on arabic to Get the timestamps back and clip them

In [ ]:
# from datasets import Dataset,Audio
# from tqdm import tqdm
# from transformers.pipelines.pt_utils import KeyDataset

# files = [...] # alist of audio paths
# dataset = Dataset.from_dict({"audio":files}).cast_column("audio",Audio(sampling_rate=16_000))

# transcriptions = []
# with torch.no_grad():
#     for tt in tqdm(model(KeyDataset(dataset,"audio"),batch_size=10,truncation=True)):
#         transcriptions.append(tt)

# Cleand Verison

In [ ]:
from transformers import pipeline
import torch
import re
import pandas as pd
from pydub import AudioSegment
import datetime
import random
import string
import tqdm
import glob
import os

completed_files = []

In [ ]:
def convert_to_hms(seconds: float) -> str:
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = round((seconds % 1) * 1000)
    output = f"{int(hours):02}:{int(minutes):02}:{int(seconds):02},{milliseconds:03}"
    return output

def convert_chunk(chunk: dict) -> str:
    start = convert_to_hms(chunk["timestamp"][0])
    end = convert_to_hms(chunk["timestamp"][1])
    text = chunk["text"].strip()
    return f"{start} --> {end}\n{text}\n\n"

In [ ]:
def find_subtitles_with_word(file_path, my_list):
    with open(file_path, "r") as file:
        content = file.read()
    subtitles = content.split("\n\n")
    found_subtitles = []

    for subtitle in subtitles:
        lines = subtitle.split("\n")
        if len(lines) >= 3:
            time_line = lines[1]
            subtitle_text = " ".join(lines[2:])

            for word in my_list:
                if word.lower() in time_line.lower() or word.lower() in subtitle_text.lower():
                    start_time, end_time = re.findall(r'\d+:\d+:\d+,\d+', time_line)
                    found_subtitles.append({
                        "start_time": start_time,
                        "end_time": end_time,
                        "subtitle": subtitle_text
                    })
    return found_subtitles

In [ ]:
def clip_audio(file_path, start_time, end_time):
    audio = AudioSegment.from_file(file_path)
    start_ms = timestamp_to_milliseconds(start_time)
    end_ms = timestamp_to_milliseconds(end_time)
    clipped_audio = audio[start_ms:end_ms]
    return clipped_audio

In [ ]:
def timestamp_to_milliseconds(timestamp):
    time_obj = datetime.datetime.strptime(timestamp, "%H:%M:%S,%f")
    milliseconds = (time_obj.minute * 60 + time_obj.second) * 1000 + int(time_obj.microsecond / 1000)
    return milliseconds

In [ ]:
# Initialize pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
model = pipeline(
    task="automatic-speech-recognition",
    model="openai/whisper-medium",
    device=device,
    chunk_length_s=30,
    generate_kwargs={"num_beams": 5}  # same as "open whisper" default
)
audio_directory_path = "/content/"
audio_formats = ["*.mp3","*.wav"]
file_path_dir = []
for audio_format in audio_formats:
    file_path_dir.extend(glob.glob(audio_directory_path+audio_format))
print(file_path_dir,"\n")


# Iterate over the audio files, transcribe, and generate subtitles
with torch.no_grad():
    for audio_file_path in file_path_dir:
        # Transcribe audio and retrieve the result
        result = model(audio_file_path, return_timestamps=True, generate_kwargs={"language": "arabic", "task": "transcribe"})

        new_file_name = os.path.basename(audio_file_path)
        file_name = new_file_name[:-4] + ".srt"
        file_path = f"/content/drive/MyDrive/ZIKIR/{file_name}"

        # Save subtitles to SRT file
        def save_subtitles_to_srt(result, file_path):
            with open(file_path, "w") as f:
                for i, chunk in enumerate(result["chunks"], start=1):
                    f.write(f"{i}\n{convert_chunk(chunk)}")



        # Save subtitles to SRT file
        save_subtitles_to_srt(result, file_path)
        print(f"Saved file: {file_path}")

        # Update the progress bar description
        print(f"Processed: {audio_file_path}")
        completed_files.extend(audio_file_path)

['/content/drive/MyDrive/audio_downloaded/qmCHCMlEvQ8.mp3', '/content/drive/MyDrive/audio_downloaded/TLpiGpcJZ8E.mp3', '/content/drive/MyDrive/audio_downloaded/4CSUrZFwVVI.mp3', '/content/drive/MyDrive/audio_downloaded/Ym4rwHzMeDU.mp3', '/content/drive/MyDrive/audio_downloaded/UPdaSdkwFKU.mp3', '/content/drive/MyDrive/audio_downloaded/RBrdl0v_anc.mp3', '/content/drive/MyDrive/audio_downloaded/hx4m6WVrWEM.mp3', '/content/drive/MyDrive/audio_downloaded/bEy4k6Au2vQ.mp3', '/content/drive/MyDrive/audio_downloaded/QVhWwlqpBwI.mp3', '/content/drive/MyDrive/audio_downloaded/HIFc-VjzHbI.mp3', '/content/drive/MyDrive/audio_downloaded/JNljUAaWw0w.mp3', '/content/drive/MyDrive/audio_downloaded/tgTiLsFlJG8.mp3', '/content/drive/MyDrive/audio_downloaded/Ic8gqtGUxbw.mp3', '/content/drive/MyDrive/audio_downloaded/PKutveeO2F0.mp3', '/content/drive/MyDrive/audio_downloaded/b5LyXG8BMlI.mp3', '/content/drive/MyDrive/audio_downloaded/1vGzj1GYdkU.mp3', '/content/drive/MyDrive/audio_downloaded/k0VcX4BvOrc.mp

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 19>:22                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/automatic_speech_recognition.py:2 │
│ 72 in __call__                                                                                   │
│                                                                                                  │
│   269 │   │   │   │   │   │   "there", "timestamps": (1.0, 1.5)}]`. The original full text can   │
│   270 │   │   │   │   │   │   `"".join(chunk["text"] for chunk in output["chunks"])`.            │
│   271 │   │   """                                                                                │
│ ❱ 272 │   │   return super().__call__(inputs, **kwargs)                                          │
│   273 │                                                                                          │
│   274 │   def _sanitize_parameters(                                                              │
│   275 │   │   self,                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1112 in __call__          │
│                                                                                                  │
│   1109 │   │   elif is_iterable:                                                                 │
│   1110 │   │   │   return self.iterate(inputs, preprocess_params, forward_params, postprocess_p  │
│   1111 │   │   elif self.framework == "pt" and isinstance(self, ChunkPipeline):                  │
│ ❱ 1112 │   │   │   return next(                                                                  │
│   1113 │   │   │   │   iter(                                                                     │
│   1114 │   │   │   │   │   self.get_iterator(                                                    │
│   1115 │   │   │   │   │   │   [inputs], num_workers, batch_size, preprocess_params, forward_pa  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/pt_utils.py:124 in __next__       │
│                                                                                                  │
│   121 │   │   │   return self.loader_batch_item()                                                │
│   122 │   │                                                                                      │
│   123 │   │   # We're out of items within a batch                                                │
│ ❱ 124 │   │   item = next(self.iterator)                                                         │
│   125 │   │   processed = self.infer(item, **self.params)                                        │
│   126 │   │   # We now have a batch of "inferred things".                                        │
│   127 │   │   if self.loader_batch_size is not None:                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/pt_utils.py:266 in __next__       │
│                                                                                                  │
│   263 │   │   │   │   │   return accumulator                                                     │
│   264 │   │                                                                                      │
│   265 │   │   while not is_last:                                                                 │
│ ❱ 266 │   │   │   processed = self.infer(next(self.iterator

In [ ]:
# Find matching subtitles with given words

directory_path = "/content/drive/MyDrive/ZIKIR/"
str_formats = ["*.srt"]
transcriped_files_path = []
for str_format in str_formats:
    transcriped_files_path.extend(glob.glob(directory_path+str_format))
print(transcriped_files_path,"\n")
words_to_search = ["محمد", "لله"]
for file_path in transcriped_files_path:
    matching_subtitles = find_subtitles_with_word(file_path, words_to_search)
    for subtitle in matching_subtitles:
        print("Start time:", subtitle["start_time"])
        print("End time:", subtitle["end_time"])
        print("Subtitle Text:", subtitle["subtitle"])
        print("---")

    # Save matching subtitles to a CSV file
    subtitle_data = []
    for subtitle in matching_subtitles:
        subtitle_data.append({
            "start_time": subtitle["start_time"],
            "end_time": subtitle["end_time"],
            "subtitle": subtitle["subtitle"],
        })

    subtitle_df = pd.DataFrame(subtitle_data)
    output_path = f"{file_path[:-3]}csv"
    subtitle_df.to_csv(output_path, index=False)
    completed_files.extend(output_path)
    print("Subtitle DataFrame saved successfully.")

In [ ]:
# Clip audio based on subtitles
audio_directory_path #path to the audio files
output_directory = "/content/drive/MyDrive/ZIKIR/cliped_audio/"

csv_path = "/content/drive/MyDrive/ZIKIR/"
file_pattern ="*.csv"
csv_files = glob.glob(csv_path+file_pattern)
print(csv_files)
for df_path in csv_files:
    subtitle_df = pd.read_csv(df_path)
    audio_file_name = os.path.splitext(os.path.basename(df_path))[0]
    wav_filename = audio_directory_path+audio_file_name + ".mp3"
    for index, row in subtitle_df.iterrows():
        start_time = row["start_time"]
        end_time = row["end_time"]
        subtitle_text = row["subtitle"]

        # Generate a unique name for the clipped audio file based on the subtitle
        unique_chars = ''.join(random.choices(string.ascii_letters + string.digits, k=4))  # Generate a random suffix
        file_name = audio_file_name + "_" + unique_chars + ".wav"
        print(wav_filename)
        # Clip the audio
        clipped_audio = clip_audio(wav_filename, start_time, end_time)

        output_path = output_directory + file_name
        clipped_audio.export(output_path, format="wav")

        print(f"Clipped audio saved: {output_path}")

In [ ]:
print(completed_files)
os.remove(f for f in completed_files)
print(f"Deleted: ",completed_files)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'completed_files' is not defined